In [1]:
import numpy as np
import scipy.io as sio
from sklearn.neural_network import MLPClassifier
# from sklearn.preprocessing import StandardScaler  

In [2]:
split_files_path = './train_data_split/'
split_file_name = 'train_feat_class_'

In [ ]:
def computeMean_std():
    X_train = sio.loadmat('train_data.mat')['train_feat']
    x_mean = X_train.mean(axis=0)
    x_std = X_train.std(axis=0)
    return x_mean, x_std

In [7]:
def createSplitFileName(index_split):
    if index_split < 10:
        return split_files_path + split_file_name+ '00'+ str(index_split) + '.mat'
    else:
        if index_split < 100:
            return split_files_path + split_file_name + '0' + str(index_split) + '.mat'
        else:
            return split_files_path + split_file_name + str(index_split) + '.mat'

In [2]:
def readSplitData(file_name,index_split):
    X_part_train = sio.loadmat(file_name)['train_feat_c']
    m, n = X_part_train.shape
    y_part_train = np.full(m,index_split)
    return X_part_train, y_part_train

In [3]:
print('------------------------导入数据集-------------------------')
X_train = sio.loadmat('X_train.mat')['X_train']
y_train = sio.loadmat('y_train.mat')['y_train']
m_x, n_x = (X_train.shape)

------------------------导入数据集-------------------------


In [4]:
def readPartialData(X_train,y_train,m_x,index, split_size):
    s1 = index * split_size
    e = s1 + split_size
    if e <= m_x:
        return X_train[s1:e,:],y_train[s1:e].ravel()
    else:
         return X_train[s1:,:],y_train[s1:].ravel()

In [10]:
mlp = MLPClassifier(solver='sgd',learning_rate_init  = 0.01, alpha=0.001, learning_rate='adaptive',max_iter=60,shuffle = True,
                    activation='relu',batch_size = 128,verbose=True,
                   hidden_layer_sizes=(1600,))

In [11]:
max_iter = 50
num_classes = 397
split_size = 4765
split_times = 16
classes = np.arange(num_classes)
# mlp.classes_ = classes
for i in range(max_iter):   
    for index in range(split_times):
        X_part_train, y_part_train = readPartialData(X_train,y_train,m_x, index,split_size)
        if index == 0:
            mlp = mlp.partial_fit(X_part_train,y_part_train,classes)
        else:
            mlp = mlp.partial_fit(X_part_train,y_part_train)
        print("iter :" + str(i) + ",split time: " + str(index) + ", accuracy", mlp.score(X_part_train,y_part_train))

Iteration 1, loss = 5.65247279
iter :0,split time: 0, accuracy 0.630640083945
Iteration 2, loss = 4.76493694
iter :0,split time: 1, accuracy 0.677649527807
Iteration 3, loss = 4.35677978
iter :0,split time: 2, accuracy 0.697166841553
Iteration 4, loss = 4.11806952
iter :0,split time: 3, accuracy 0.741028331584
Iteration 5, loss = 3.96035540
iter :0,split time: 4, accuracy 0.735571878279
Iteration 6, loss = 3.76754606
iter :0,split time: 5, accuracy 0.753830010493
Iteration 7, loss = 3.73228719
iter :0,split time: 6, accuracy 0.738090241343
Iteration 8, loss = 3.63614534
iter :0,split time: 7, accuracy 0.764113326338
Iteration 9, loss = 3.55308573
iter :0,split time: 8, accuracy 0.758656873033
Iteration 10, loss = 3.54495852
iter :0,split time: 9, accuracy 0.760545645331
Iteration 11, loss = 3.39960478
iter :0,split time: 10, accuracy 0.778384050367
Iteration 12, loss = 3.41935019
iter :0,split time: 11, accuracy 0.781532004197
Iteration 13, loss = 3.36906368
iter :0,split time: 12, acc

iter :6,split time: 5, accuracy 0.999370409234
Iteration 103, loss = 0.05817756
iter :6,split time: 6, accuracy 0.999580272823
Iteration 104, loss = 0.05729474
iter :6,split time: 7, accuracy 0.999370409234
Iteration 105, loss = 0.05757797
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Setting learning rate to 0.000003
iter :6,split time: 8, accuracy 0.999580272823
Iteration 106, loss = 0.05516307
iter :6,split time: 9, accuracy 0.999580272823
Iteration 107, loss = 0.05235058
iter :6,split time: 10, accuracy 0.999790136411
Iteration 108, loss = 0.05284993
iter :6,split time: 11, accuracy 0.999790136411
Iteration 109, loss = 0.05098786
iter :6,split time: 12, accuracy 1.0
Iteration 110, loss = 0.05087612
iter :6,split time: 13, accuracy 0.999790136411
Iteration 111, loss = 0.06540959
iter :6,split time: 14, accuracy 0.999370409234
Iteration 112, loss = 0.06275885
iter :6,split time: 15, accuracy 1.0
Iteration 113, loss = 0.05931273
Training loss did not

C:\Users\jiamoufang\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

In [12]:
print (mlp.score(X_train,y_train))
print (mlp.n_layers_)
print (mlp.n_iter_)
print (mlp.loss_)
print( mlp.out_activation_)

0.999619622246
3
146
0.0614764197436
softmax


In [13]:
import h5py
def loadTestData(path):
    features = {}
    with h5py.File(path, 'r') as f:
#         print(list(f.keys()))
        features = f['test_feat']
        features = np.mat(features).T
        print(features.shape)
    return features

In [ ]:
import csv
def save_result(file_name, result):
    '''保存最终的预测结果
    input:  file_name(string):保存最终结果的文件名
            result(mat):最终的预测结果
    '''
    with open(file_name) as csvFile:
        rows = csv.reader(csvFile)
        with open("newest_7_17_n1600.csv",'w', newline='') as f:
            writer = csv.writer(f)
            for index,row in enumerate(rows):
                tmp_row = row
                if index >= 1:
                    # tmp_row[-1] = int(result[index-1,0])
                    tmp_row[-1] = int(result[index-1])
                writer.writerow(tmp_row)

In [ ]:

print("------------------开始测试---------------") 
file_test = 'test_data_raw.mat'
file_out = 'submission_sample.csv'

test_feat = loadTestData(file_test)
test_mean = test_feat.mean(axis=0)
test_std = test_feat.std(axis=0)
test_feat_scaled = (test_feat-test_mean)/test_std
p = mlp.predict(test_feat_scaled)
print("--------------------保存结果----------------")
print(p.shape)
save_result(file_out,np.mat(p).T)

------------------开始测试---------------
(19850, 6812)


In [ ]:
print(p.shape)